# Dereplicator evaluation
This notebook takes the SMILES from the spectral library hits and from dereplicator TOP1 result, and compare them against each other (tanimoto from the fingerprint).

Workflow:
get table / get smiles / make mol obj / output smiles isomeric / make mol obj / make fingerprint / compare fingerprint / add the fingerprint in the table

In [26]:
from rdkit import Chem
import pandas as pd
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.Fingerprints import *
from rdkit import DataStructs
import itertools

In [27]:
#Import Table
df = pd.read_csv("Table_S7.csv",  sep=';', header=0)
df.head(2)

,Name,Mass,num-amide,num-OC,num-NC,#theo,#exper,TopID,Derep+ scr,Derep+ rank,...,Derep-g rank,Derep scr,Derep rank,#matches,MF1,#isomers,dnp-class,SMILES1,MF2,SMILES2
0,Puwainaphycin_B_M+H,1232.74,10.0,4.0,24.0,260.0,158.0,Puwainaphycin_B_M+H,48.0,1.0,...,7.0,24.0,1.0,145.0,C59H100N12O16,3.0,aminoacids/peptides,NC(C[C@H]1C(N2[C@H](C(N[C@@H]([C@@H](O)C(N[C@@...,C59H100N12O16,[H]O[C@@]([H])(C([H])([H])[H])[C@]1([H])N([H])...
1,Cyclosporin_A_M+H,1201.84,11.0,1.0,29.0,272.0,81.0,Cyclosporin_A_M+H,48.0,1.0,...,10.0,29.0,1.0,143.0,C62H111N11O12,6.0,none,CCC1NC(=O)C(C(O)C(C)C\C=C\C)N(C)C(=O)C(C(C)C)N...,C62H111N11O12,[H]OC([H])(C([H])(C([H])([H])[H])C([H])([H])C(...


In [28]:
#Make list from SMILES column(s)
List_smiles_1 = []
List_smiles_1 = df['SMILES1'].tolist()
List_smiles_2 = []
List_smiles_2 = df['SMILES2'].tolist()
len(List_smiles_1)

5473

In [29]:
# Make a list of mol object from smiles list
mol1_list = []
mol2_list = []

#Run for loops for mol object from smiles list
for x in List_smiles_1:
    try:
        mol1_list.append(Chem.MolFromSmiles(x))
    except:
        pass
        mol1_list.append('')
    
#Run for loops for mol object from smiles list
for x in List_smiles_2:
    try:
        mol2_list.append(Chem.MolFromSmiles(x))
    except:
        pass
        mol2_list.append('')


In [30]:
len(mol2_list)

5473

In [31]:
#Convert back to isomeric smiles list
smiles1_list_iso = []
smiles2_list_iso = []

#Run for loops to isomeric smiles
for x in mol1_list:
    try:
        smiles1_list_iso.append(Chem.MolToSmiles(x, isomericSmiles=False))
    except:
        pass
        smiles1_list_iso.append('')
    
#Run for loops to isomeric smiles
for x in mol2_list:
    try:
        smiles2_list_iso.append(Chem.MolToSmiles(x, isomericSmiles=False))
    except:
        pass
        smiles2_list_iso.append('')


In [32]:
len(smiles2_list_iso)

5473

In [33]:
#Back to mol object
mol1_list_iso = [Chem.MolFromSmiles(x) for x in smiles1_list_iso]
mol2_list_iso = [Chem.MolFromSmiles(x) for x in smiles2_list_iso]
len(mol1_list_iso)

5473

In [34]:
# Create list
fp1_list_iso = []
fp2_list_iso = []

#Run for loops for fingerprint from mol1
for x in mol1_list_iso:
    try:
        fp1_list_iso.append(FingerprintMols.FingerprintMol(x))
    except:
        pass
        fp1_list_iso.append('')
    
#Run for loops for fingerprint from mol2
for x in mol2_list_iso:
    try:
        fp2_list_iso.append(FingerprintMols.FingerprintMol(x))
    except:
        pass
        fp2_list_iso.append('')

In [35]:
#Chem.MolToSmiles(mol1_list_iso[110])

In [36]:
#make a class for fingerprint from mol pair
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.Fingerprints import *
from rdkit import DataStructs
import itertools

def fingerprintingbro (list1, list2):
    for x,y in itertools.izip(list1,list2):
        results.append(DataStructs.FingerprintSimilarity(x,y))

In [37]:
# Run the function for fingerprint
results = []
fingerprintingbro(fp1_list_iso,fp2_list_iso)
TOTAL = len(results)
TOTAL

5473

In [38]:
# Number of correct annotation
TOP1 = results.count(1.0)
TOP1

1210

In [39]:
# Change to float
results = map(float, results)

In [40]:
df.shape

(5473, 21)

In [41]:
# Get MF into list
MF1 = df['MF1'].tolist()
MF2 = df['MF2'].tolist()

# Compare MF
MFagreement = []

for x,y in itertools.izip(MF1,MF2):
    if x == y:
        MFagreement.append('Yes')
    else:
        MFagreement.append('No')
len(MFagreement)

5473

In [42]:
#Number of correct MF
CORRECT_MF = MFagreement.count('Yes')
CORRECT_MF

1626

In [43]:
# Make the series from list
se = pd.Series(results)
df['fingerprint'] = se.values

se2 = pd.Series(MFagreement)
df['MFagreement'] = se2.values

se3 = pd.Series(smiles1_list_iso)
df['SMILES1_FP1'] = se3.values

se4 = pd.Series(smiles2_list_iso)
df['SMILES2_FP2'] = se4.values

In [44]:
df.to_csv('dereplicator_results_with_fingerprint.tsv', sep = '\t', index = False)

In [45]:
#STATS CORRECT MF
from __future__ import division
CORRECT_MF / TOTAL

0.29709482916133745

In [46]:
#STATS TOP1
TOP1 / TOTAL

0.22108532797368902

In [134]:
#Subset correct annotation derep+ at score > 8 
df2= df.loc[df['Derep+ scr'] > 8 ]
total_above_score = len(df2)
total_correct_above_score = len(df2.loc[df2['fingerprint'] == 1 ])

#Correct annotation at derep+ score threshold
total_correct_above_score / total_above_score 

0.7857142857142857

In [135]:
#Total annotation above derep+ score treshold > 8
total_above_score

364

In [157]:
#Subset of annotation derep+ at score > 8 and fingerprint 0.7
df2= df.loc[df['Derep+ scr'] > 8]
total_above_score = len(df2)
total_correct_above_score = len(df2.loc[df2['fingerprint'] > 0.7 ])

#Correct annotation at derep+ score threshold
total_correct_above_score / total_above_score 

0.8983516483516484

In [158]:
#Total annotation above derep+ score treshold and fingerprint 0.7
total_above_score

364

In [160]:
#Subset of annotation derep+ at score > 8 and 0.7 < fingerprint < 1
df2= df.loc[df['Derep+ scr'] > 8]
df2= df2.loc[df['fingerprint'] < 1]
total_above_score = len(df2)
total_correct_above_score = len(df2.loc[df2['fingerprint'] > 0.7 ])

#Correct annotation at derep+ score threshold
total_correct_above_score / total_above_score 

0.5256410256410257

In [165]:
#Total annotation above derep+ score treshold 0.7 < fingerprint < 1
total_above_score

1210

In [167]:
df2= df2.loc[df['fingerprint'] == 1]
total_above_score = len(df2)
total_correct_above_score = len(df2.loc[df2['derepl scr'] > 0.7 ])

KeyError: 'derepl scr'